References:

https://saturncloud.io/blog/how-to-apply-resnet-on-cifar10-after-resizing-using-pytorch/

https://github.com/kuangliu/pytorch-cifar/blob/master/main.py


In [1]:
import torch
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
from torch.cuda.amp import autocast, GradScaler

import torch.nn as nn
import torch.optim as optim

In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
# transform = transforms.Compose(
#     [transforms.ToTensor(),
#     transforms.Resize(128),
#     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor()
])

batch_size = 128

# Create train and test data
train_data = CIFAR10(root='./data', train=True, download=True, transform=transform)
test_data = CIFAR10(root='./data', train=False, download=True, transform=transform)

# Create dataloaders
train_dl = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4)
test_dl = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=4)

# Report split sizes
print('Training set has {} instances'.format(len(train_dl)))
print('Testing set has {} instances'.format(len(test_dl)))

Files already downloaded and verified
Files already downloaded and verified
Training set has 391 instances
Testing set has 79 instances


In [4]:
# def train(model, train_loader, criterion, optimizer, device, scaler):
#     model.train()
#     for inputs, labels in train_loader:
#         inputs, labels = inputs.to(device), labels.to(device)
#         optimizer.zero_grad()

#         with autocast():
#             outputs = model(inputs)
#             loss = criterion(outputs, labels)

#         scaler.scale(loss).backward()
#         scaler.step(optimizer)
#         scaler.update()
#     print("Finished training")

# Training
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    for _, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    accuracy = 100 * correct / total
    print('Accuracy on train set: {:.2f}%'.format(accuracy))

# def test(model, test_loader, device):
#     model.eval()
#     correct = 0
#     total = 0
#     with torch.no_grad():
#         for inputs, labels in test_loader:
#             inputs, labels = inputs.to(device), labels.to(device)
#             outputs = model(inputs)
#             _, predicted = torch.max(outputs.data, 1)
#             total += labels.size(0)
#             correct += (predicted == labels).sum().item()
#     accuracy = 100 * correct / total
#     print('Accuracy on test set: {:.2f}%'.format(accuracy))

def test(model, test_loader, criterion, device):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for _, (inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    accuracy = 100 * correct / total
    print('Accuracy on test set: {:.2f}%'.format(accuracy))

In [5]:
resnet = models.resnet18(pretrained=False)

# resnet = ResNet18()
# resnet.fc = nn.Linear(resnet.fc.in_features, 10)
resnet.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)

scaler = GradScaler()  # For mixed precision training

# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

c:\Users\Jacqu\miniconda3\envs\conda-torch\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Jacqu\miniconda3\envs\conda-torch\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [9]:
import time

# Define the maximum training duration (in seconds)
max_training_duration = 30 * 60  # 30 minutes in seconds

start_time = time.time()

for epoch in range(10):
    print(f'Epoch: {epoch}')
    with torch.autocast(device_type='cuda', dtype=torch.float16):
        train(resnet, train_dl, criterion, optimizer, device)
        # test(resnet, test_dl, criterion, device)
        # scheduler.step()

    # Check if the training duration exceeds the maximum duration
    if time.time() - start_time > max_training_duration:
        print("Training stopped due to time limit.")
        break

        # # Runs the forward pass with autocasting.
        # with autocast(device_type='cuda', dtype=torch.float16):
        #     output = model(input)
        #     loss = loss_fn(output, target)

        # # Scales loss.  Calls backward() on scaled loss to create scaled gradients.
        # # Backward passes under autocast are not recommended.
        # # Backward ops run in the same dtype autocast chose for corresponding forward ops.
        # scaler.scale(loss).backward()

        # # scaler.step() first unscales the gradients of the optimizer's assigned params.
        # # If these gradients do not contain infs or NaNs, optimizer.step() is then called,
        # # otherwise, optimizer.step() is skipped.
        # scaler.step(optimizer)

        # # Updates the scale for next iteration.
        # scaler.update()

Epoch: 0
Accuracy on train set: 0.00%
Epoch: 1
Accuracy on train set: 10.11%
Epoch: 2
Accuracy on train set: 8.45%
Epoch: 3


KeyboardInterrupt: 